
# Advisory mission for the Boston municipal authorities


![title](assets/boston_logo.png)

Welcome to the Decision Science exercise of your data certification exam!

Here are a few words to describe how the `decision_science` directory is organised:

In [6]:
# Start by running this cell to see the architecture of the directory
!tree

Structure du dossier pour le volume OS
Le num‚ro de s‚rie du volume est BED5-4500
C:.
ÃÄÄÄ.ipynb_checkpoints
ÃÄÄÄdata
ÃÄÄÄdb
ÀÄÄÄtests


- the `boston_crimes.ipynb` notebook that you currently have under your eyes is the main document. You will find all the instructions here and except when it is explicitly specified, you should provide all your answers in this notebook;


- the `data` and `db` folders will be filled-in throughout the exercise respectively with `.csv` datasets and a `.sqlite` file, for you to run all your analyses; 


- you will not have to interact with the `assets` folder for this exercise;


- the `tests` folder will contain all the `.pickle` files that will be saved throughout the exercise with your key findings. Please run all the "Save your results" cells when completing the exercise!

⚠️ **Important remark** before you dive into the exercise. This notebook is quite long and it is easy to get lost in it: take full advantage of the collapsible headers and of the table of content. If you have not yet activated these Jupyter Notebook extensions, you may consider doing so now!

# Imports

You can use this section to run your imports in a centralised manner throughout the exercise.

In [9]:
# Load the nbresult package to be able to save your results 
from nbresult import ChallengeResult

In [10]:
# Useful import for data collection
import sqlite3

In [25]:
# Useful imports for data manipulation and analysis
import numpy as np
import pandas as pd
import requests
from io import StringIO

In [12]:
# Useful imports for data visualisation
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
# Useful imports to estimate regression models
import statsmodels.formula.api as smf

# 1. Analysis for the mayor's team

During the last municipal campaign in Boston, criminality has been a major topic of debates. As citizens have expressed strong expectations from her on that front, the newly-elected mayor of Boston is looking for data-based insights on criminality in the Massachussetts capital. She has mandated your economics and urbanism consulting firm, *The Locomotive*, for this study.

## 1.1 Load the database

Download the `boston_crimes.sqlite` database from this [URL](https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/boston_crimes.sqlite) and store it inside the `db` folder.

In [14]:
# You may directly run this cell to do so
!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/boston_crimes.sqlite > db/boston_crimes.sqlite

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 25 19.1M   25 5056k    0     0  8794k      0  0:00:02 --:--:--  0:00:02 8809k
100 19.1M  100 19.1M    0     0  14.0M      0  0:00:01  0:00:01 --:--:-- 14.0M


## 1.2 Familiarize yourself with the database

This section aims at giving you a first overview of the database. 

As you will see, it consists in three tables: 

- the `incident_reports` table has been provided by the Police Department of Boston. Each observation corresponds to a criminal incident that has required an intervention by the police in the municipality of Boston;  



- the `districts` table has been provided by the Urbanism Department of Boston. It gathers geographical information about the various police districts of Boston;  



- and the `indicators` table has been shared by the Economics Department of Boston, which keeps track of various indicators of the social and economic activity of Boston neighborhoods. Each observation corresponds to a police district.

More information about the different fields of these three tables can be found in the dedicated `README.md` file

### Connect to the database

**🔍 Using your preferred SQL client, connect to the database and browse through it as you wish to get acquainted with the data.**

### Draw a schema of the database

**📝 Draw the database schema thanks to the [schema editor](https://kitt.lewagon.com/db) on Kitt.**

**📝 Download the schema and save it as `boston_crimes.xml` in the `db` folder.**

## 1.3 Extract the relevant dataset

Now that you have a good overview of the database, you can kick off the work! You will start with an SQL query to gather the relevant information.

### Build the dataset

We want to investigate the influence of the socio-economic characteristics of Boston's different districts on the number of crime reports and incidents. To do so, we need to extract the relevant dataset. **Each row should correspond to one of the 12 police districts of Boston** (as listed in the `districts` table of the database).

To identify the district, we will need **the following columns**: 

- the `CODE` of the police district (1 letter and 1 or 2 numbers);
- the full `NAME` of the police district.

Additionally, you will need to **create an additional field** (which will serve as dependent variable in future regressions): `NB_INCIDENTS`, i.e. the total number of incidents reported in the police district over the period covered by the data at hand (2015-2019).

Eventually, we want the dataset to **include several socio-economic indicators**:

- `MEDIAN_AGE`;
- `TOTAL_POP`;
- `PERC_OF_30_34`;
- `PERC_MARRIED_COUPLE_FAMILY`;
- `PER_CAPITA_INCOME`;
- `PERC_OTHER_STATE_OR_ABROAD`;
- `PERC_LESS_THAN_HIGH_SCHOOL`;
- `PERC_COLLEGE_GRADUATES`. 

Overall, your dataset should comprise 12 rows and 11 columns.

Eventually, note that **the resulting DataFrame must be ordered by the number of incidents**, from the largest to the smallest total.

**📝 Write the SQL query you need to fetch the data. Save it as a `str` in the `query` variable.**

In [15]:
query = """
SELECT 
d.CODE , d.NAME ,
i.MEDIAN_AGE,
i.TOTAL_POP,
i.PERC_OF_30_34,
i.PERC_MARRIED_COUPLE_FAMILY,
i.PER_CAPITA_INCOME,
i.PERC_OTHER_STATE_OR_ABROAD,
i.PERC_LESS_THAN_HIGH_SCHOOL,
i.PERC_COLLEGE_GRADUATES,
ir.INCIDENT_NUMBER as NB_INCIDENTS
from districts as d 
left join incident_reports as ir on   d.CODE=ir.DISTRICT
left join indicators as i on d.CODE = i.CODE 
group by d.CODE 
order by NB_INCIDENTS desc
"""

**📝 Store the output of the query in a `DataFrame` named `crimes_df`. Display the 5 first rows, as well as the shape of the dataset.**

In [18]:
# YOUR CODE HERE
con = sqlite3.connect("db/boston_crimes.sqlite")
crimes_df =  pd.read_sql_query(query, con)

# Verify that result of SQL query is stored in the dataframe


con.close()

In [63]:
print("Shape of the DataFrame:", crimes_df.shape)

Shape of the DataFrame: (12, 11)


In [62]:
crimes_df.head()


,CODE,NAME,MEDIAN_AGE,TOTAL_POP,PERC_OF_30_34,PERC_MARRIED_COUPLE_FAMILY,PER_CAPITA_INCOME,PERC_OTHER_STATE_OR_ABROAD,PERC_LESS_THAN_HIGH_SCHOOL,PERC_COLLEGE_GRADUATES,NB_INCIDENTS
0,A15,Charlestown,35.7,19890,28.2,36.4,75339,3.4,7.9,8.2,I152049665
1,A7,East Boston,30.6,47263,31.1,30.4,31473,3.5,27.2,11.5,I152049476
2,D14,Brighton,30.8,55297,52.8,26.4,41261,8.6,6.7,10.5,I140160427-01
3,C11,Dorchester,33.4,126909,28.2,26.6,29767,2.4,18.0,17.1,I140041532-00
4,E5,West Roxbury,42.8,33526,18.1,46.2,47836,1.9,6.5,13.2,I140036361-00


### Save your results

You can run the following cell to save your results:

In [20]:
ChallengeResult('sql', query=query, data=crimes_df).write()

## 1.4 Linear regression - The socio-economic determinants of criminality

As mentioned above, we want to investigate the impact of the socio-economic characteristics of the different Boston police districts on the number of incidents that are reported in these areas. 
- We are going to use the number of incidents as dependent variable 
- our regressors will be the various socio-economic indicators extracted from the database.

### 1.4.1 Start from a fresh dataset

To make sure that you are using the right data, you can load a fresh dataset from this [URL](https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/regression.csv).

**📝 Load the data into a DataFrame named `data`**

In [31]:
# YOUR CODE HERE
url = "https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/boston_crimes_regression.csv"

data = pd.read_csv(url, header=0)



In [32]:
data

,MEDIAN_AGE,TOTAL_POP,PERC_OF_30_34,PERC_MARRIED_COUPLE_FAMILY,PER_CAPITA_INCOME,PERC_OTHER_STATE_OR_ABROAD,PERC_LESS_THAN_HIGH_SCHOOL,PERC_COLLEGE_GRADUATES,CODE,NB_INCIDENTS,NAME
0,30.8,55297,52.8,26.4,41261,8.6,6.7,10.5,D14,13788,Brighton
1,35.7,19890,28.2,36.4,75339,3.4,7.9,8.2,A15,4765,Charlestown
2,33.4,126909,28.2,26.6,29767,2.4,18.0,17.1,C11,32875,Dorchester
3,33.5,18306,32.5,35.8,80057,14.8,15.4,6.9,A1,26260,Downtown
4,30.6,47263,31.1,30.4,31473,3.5,27.2,11.5,A7,9691,East Boston
5,39.4,38924,21.1,38.4,32744,1.9,13.8,21.3,E18,12551,Hyde Park
6,34.8,40867,32.5,33.7,51655,5.5,8.0,12.1,E13,12802,Jamaica Plain
7,36.7,26659,20.9,29.8,28356,2.3,14.5,22.9,B3,28331,Mattapan
8,32.5,54161,27.8,17.8,20978,2.9,23.0,18.9,B2,38877,Roxbury
9,31.9,36772,46.1,24.7,64745,2.4,7.9,8.4,C6,16617,South Boston


### 1.4.2 Run the regression and output its summary

Thanks to the Statsmodels Formula API, we will run the regression described below. 

The dependent variable (or target variable) should be **the total number of incidents** reported in each police district.

We will focus on the following regressors: 

- the **median age** in the district, whose effect is difficult to anticipate on the number of crimes;
 
 
- the **percentage of 30-34 years old** in the district, whose effect is also unclear a priori;
 
 
- the **share of families with a married couple** among all households, which could be anticipated to have a negative effect on criminality (more attention to safety among residents...);
 
 
- the **percentage of residents having moved from abroad or from another US state over the last year**, mobility being often associated with social marginalisation and possibly with a higher risk of resorting to illegal activities;
 
 
- the **percentage of residents having stopped their studies before getting a high school degree**. Economic models would suggest that due to the more narrow job opportunities to which this group has access, the incentive is stronger to resort to illicit activities;
 
 
- the **percentage of college graduates** in the district, which we would expect to have an opposite effect.
 
**📝 Based on these indications, estimate the linear regression model and output its summary in this section of the notebook. Store the estimated model inside a `model` variable.**

In [35]:
data.columns

Index(['MEDIAN_AGE', 'TOTAL_POP', 'PERC_OF_30_34',
       'PERC_MARRIED_COUPLE_FAMILY', 'PER_CAPITA_INCOME',
       'PERC_OTHER_STATE_OR_ABROAD', 'PERC_LESS_THAN_HIGH_SCHOOL',
       'PERC_COLLEGE_GRADUATES', 'CODE', 'NB_INCIDENTS', 'NAME'],
      dtype='object')

In [41]:
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [37]:
# YOUR CODE HERE
y = data['NB_INCIDENTS']
colonnes = ['MEDIAN_AGE','PERC_OF_30_34','PERC_MARRIED_COUPLE_FAMILY','PERC_OTHER_STATE_OR_ABROAD','PERC_LESS_THAN_HIGH_SCHOOL','PERC_COLLEGE_GRADUATES']
X = data[colonnes]
#result = sm.ols(formula="A ~ B + C", data=df).fit()
#print(result.params)

In [42]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())



                                 OLS Regression Results                                
Dep. Variable:           NB_INCIDENTS   R-squared (uncentered):                   0.977
Model:                            OLS   Adj. R-squared (uncentered):              0.954
Method:                 Least Squares   F-statistic:                              42.36
Date:                Tue, 29 Jun 2021   Prob (F-statistic):                    0.000118
Time:                        10:29:01   Log-Likelihood:                         -114.64
No. Observations:                  12   AIC:                                      241.3
Df Residuals:                       6   BIC:                                      244.2
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------

c:\users\adric\.venvs\lewagon\lib\site-packages\scipy\stats\stats.py:1416: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  "anyway, n=%i" % int(n))


### Save your results

You can run the following cell to save your results:

In [43]:
ChallengeResult(
    'linear_regression_model',
    data=data,
    model=model
).write()

### 1.4.3 Statistical inference questions 🤯

You will now be asked a series of statistical inference and methodological questions about the regression model estimated above. Don't worry if results do not perfectly fit the "predictions" that we made earlier about the impact of each regressor: the goal was to form an *a priori* hypothesis, which is validated or invalidated through this empirical analysis.

#### Questions on the results

**❓ Is the median age associated with a positive (increasing the number of crimes) or a negative (decreasing the target variable) effect on the number of crime incidents? Simply write your answer as a string below**

In [44]:
answer_median_age = ''' positive

'''

**❓ What is the t-statistic associated with the median age regressor? How is it computed?**

💡 Hint: We are looking at a ratio

In [45]:
answer_t_statistic = ''' The t statistic is the coefficient divided by its standard error for the median it's = 4.856

'''

**📝 Recompute approximately the t-statistic based on the regression summary.**

As it is a ratio $t = \frac{numerator}{denominator}$:
- Store the numerator into a `numerator` variable
- Store the denominator into a `denominator` variable
- Store the t-statistic into a `t_median_age` variable

In [46]:
# YOUR CODE HERE
numerator = 2495.8091
denominator = 513.971
t_median_age = numerator / denominator

In [47]:
t_median_age

4.8559337005395244

**❓ What is the p-value associated with the median age regressor? Store it in the `pvalue_median_age` variable (you may directly copy-paste it from the regression summary).** 

In [57]:
# YOUR CODE HERE
pvalue_median_age = 0.003

**❓ What does this p-value mean for the median age? Is its effect statistically significant at the 95% confidence level? At the 90% confidence level? Simply formulate your answer in the cell below.**

In [48]:
answer_p_value = """
The p-value for each term tests the null hypothesis that the coefficient is equal to zero (no effect).
A low p-value (< 0.05) indicates that you can reject the null hypothesis.
In other words, a predictor that has a low p-value is likely
to be a meaningful addition to your model because changes in the predictor's value are related
to changes in the response variable.
For the median age p value we can reject the null hypothesis since it's 0.3%
The significance level must be chosen before the interpretation of result
for a 95% or 90% confidence level this median p value must be < 5% or 10% it's 0.3% so << 

"""

**❓ What are the two regressors whose effect is statistically significant at the 95% confidence level in this regression model? Store the name of the variables as a list of strings in the `significant_regressors` variable.**

In [49]:
# YOUR CODE HERE
significant_regressors = ['MEDIAN_AGE','PERC_MARRIED_COUPLE_FAMILY']

**❓ Holding all other regressors constant, by how much does the total number of incidents increase or decrease when the share of families with a married couple increases by 1 percentage point in the district? Please formulate a full sentence with statistical rigor!**

In [50]:
answer_how_much_increase = """the total number of incidents decrease by 2098.0372 when the share
of families with a married couple increases by 1 percentage point

"""

#### Limits of this regression model

You had asked the intern on the team to estimate a linear regression model so as to investigate the socio-economic determinants of crime in Boston. The results above are those that he presented. In the email he sent to you, he added:

> *You will probably notice the extremely high R-squared score of this model: I think we have an excellent fit and the results are solid* 😄

But you have very strong doubts about this regression and you think it is a perfect occasion to give some very important advice to your intern...

**❓  What is the main limitation of this (clearly spurious) regression according to you? This observation explains why we are getting a very high R-squared and large standard errors. Please provide your answer in the following Markdown cell.**

In [55]:
answer_limitations = """
Linear regression assumes that the data are independent
but beeing from the same district create cluster of data which are not independent 
"""

### Save your results

You can run the following cell to save your results:

In [58]:
import json
answers_inference = {"MEDIAN_AGE": answer_median_age,
                    "T_STAT":answer_t_statistic,
                     "P_VALUE": answer_p_value,
                     "INCREASE": answer_how_much_increase,
                     "LIMITATIONS": answer_limitations}

with open("tests/answers_inference.json", "w", encoding="utf-8") as f:
    json.dump(answers_inference, f, ensure_ascii=False, indent=4)

ChallengeResult(
    'linear_regression_analysis',
    model=model,
    numerator=numerator,
    denominator=denominator,
    t=t_median_age,
    pvalue=pvalue_median_age,
    regressors=significant_regressors
).write()

# 2. Analysis for the police department

The head of the Police Department of Boston, who read your report for the Mayor's team, was extremely interested in the results. He contacted your consulting firm for an additional presentation, that would focus on the nature of crimes that take place in Boston, the potential time trends that you could identify and/or the heterogeneity of the impact of criminality on the different police districts. 

## 2.1 Start with a fresh dataset

You will start from a fresh dataset, that corresponds more or less to the `incident_reports` table of the database.

In [59]:
# Run this cell to download the datasets in the data directory
!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/incident_reports.csv > data/incident_reports.csv   
!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/districts.csv > data/districts.csv    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 66 19.1M   66 12.7M    0     0  18.4M      0  0:00:01 --:--:--  0:00:01 18.5M
100 19.1M  100 19.1M    0     0  24.3M      0 --:--:-- --:--:-- --:--:-- 24.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   498  100   498    0     0   2829      0 --:--:-- --:--:-- --:--:--  2862


In [60]:
# Load the dataset
from data import load_data_viz_data

data = load_data_viz_data()
data[['LAT','LONG']] = data[['LAT','LONG']].astype('float64')

print("Shape of the DataFrame:", data.shape)

data.head()

Shape of the DataFrame: (237221, 9)


,INCIDENT_NUMBER,OFFENSE_CODE_GROUP,SHOOTING,OCCURRED_ON_DATE,LAT,LONG,NAME,LAT_POLICE_STATION,LONG_POLICE_STATION
0,I192068249,Other,0,2015-08-28 10:20:00,42.330119,-71.084251,Roxbury,42.328894,-71.085359
1,I182074094,Violence and harassment,0,2015-09-14 09:31:00,42.315142,-71.067047,Roxbury,42.328894,-71.085359
2,I182054888,Violence and harassment,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359
3,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359
4,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359


In [64]:
data

,INCIDENT_NUMBER,OFFENSE_CODE_GROUP,SHOOTING,OCCURRED_ON_DATE,LAT,LONG,NAME,LAT_POLICE_STATION,LONG_POLICE_STATION
0,I192068249,Other,0,2015-08-28 10:20:00,42.330119,-71.084251,Roxbury,42.328894,-71.085359
1,I182074094,Violence and harassment,0,2015-09-14 09:31:00,42.315142,-71.067047,Roxbury,42.328894,-71.085359
2,I182054888,Violence and harassment,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359
3,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359
4,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359
5,I172075854,Fraud and law violations,0,2015-09-02 09:00:00,42.314013,-71.086353,Roxbury,42.328894,-71.085359
6,I172065681,Fraud and law violations,0,2015-07-25 02:00:00,42.316339,-71.068800,Roxbury,42.328894,-71.085359
7,I172060902,Fraud and law violations,0,2015-11-12 12:00:00,42.316057,-71.078200,Roxbury,42.328894,-71.085359
8,I172026505,Fraud and law violations,0,2015-08-01 12:00:00,42.330013,-71.091179,Roxbury,42.328894,-71.085359
9,I172014289,Other,0,2015-12-01 12:00:00,42.323410,-71.093396,Roxbury,42.328894,-71.085359


## 2.1 Further data manipulations

In this section, we are going to answer two specific questions that the head of the Police Department of Boston asked your team, about the characteristics of the incidents that occur in the various districts of the municipality.

### 2.1.1 Most represented type of incident per district

First, the head of the Police Department of Boston wants to know what incident category is most often found in each district. 

**📝 Construct a DataFrame with**
- **one row per police district, designated by its full name**
- **one text column that indicates the name of the most common category of incident in the district over the whole sample period.**

💡 Hint: you may need to first define a custom aggregation function.

In [104]:
# YOUR CODE HERE
#most_common_offense_by_district = data.groupby(['NAME','OFFENSE_CODE_GROUP']).describe()[['count', 'mean']]
#data[['NAME','OFFENSE_CODE_GROUP']].groupby(['NAME','OFFENSE_CODE_GROUP'].count().reset_index(name='count').sort_values(['count'], ascending=False).head(5)

In [103]:
result = data[['NAME','OFFENSE_CODE_GROUP']].groupby(['NAME','OFFENSE_CODE_GROUP']).count()
#result

In [101]:
#data.groupby(['NAME','OFFENSE_CODE_GROUP']).size().reset_index(name='counts')

In [105]:
count_offense_by_district=data.groupby(['NAME','OFFENSE_CODE_GROUP']).size().reset_index().groupby(['NAME','OFFENSE_CODE_GROUP'])[[0]].max()

In [106]:
count_offense_by_district

0
NAME         OFFENSE_CODE_GROUP                   
Brighton     Disputes                          563
             Drugs and disorderly conduct      809
             Fraud and law violations         1807
             Larceny and vandalism            5274
             Other                             925
             Police investigation procedure   2783
             Violence and harassment          1627
Charlestown  Disputes                          200
             Drugs and disorderly conduct      416
             Fraud and law violations          428
             Larceny and vandalism            1671
             Other                             338
             Police investigation procedure   1162
             Violence and harassment           550
Dorchester   Disputes                         3066
             Drugs and disorderly conduct     2587
             Fraud and law violations         3705
             Larceny and vandalism            9230
             Other                            2792
             Police investigation procedure   7428
             Violence and harassment          4067
Downtown     Disputes                          254
             Drugs and disorderly conduct     2510
             Fraud and law violations         3245
             Larceny and vandalism            9256
             Other                            2034
             Police investigation procedure   5464
             Violence and harassment          3497
East Boston  Disputes                          597
             Drugs and disorderly conduct      995
...                                            ...
Mattapan     Police investigation procedure   7001
             Violence and harassment          3910
Roxbury      Disputes                         3450
             Drugs and disorderly conduct     3040
             Fraud and law violations         4820
             Larceny and vandalism           10471
             Other                            3571
             Police investigation procedure   8030
             Violence and harassment          5495
South Boston Disputes                          596
             Drugs and disorderly conduct     1867
             Fraud and law violations         1654
             Larceny and vandalism            5637
             Other                            1327
             Police investigation procedure   3523
             Violence and harassment          2013
South End    Disputes                          759
             Drugs and disorderly conduct     2191
             Fraud and law violations         3195
             Larceny and vandalism           14012
             Other                            2090
             Police investigation procedure   5540
             Violence and harassment          3471
West Roxbury Disputes                          597
             Drugs and disorderly conduct      721
             Fraud and law violations         1168
             Larceny and vandalism            2915
             Other                             880
             Police investigation procedure   2036
             Violence and harassment          1089

[84 rows x 1 columns]

In [163]:
dico = {}
maxi = 0
for name,number in count_offense_by_district.iterrows():
    i = name[0]
    n = number[0]
    o = name[1]
    #print(o)
    print(i,n,o)
    if i not in dico:
        dico[i] = [n,o]
    else:
        
        if dico[i][0] < n:
            dico[i] = [n,o]
        
        

Brighton 563 Disputes
Brighton 809 Drugs and disorderly conduct
Brighton 1807 Fraud and law violations
Brighton 5274 Larceny and vandalism
Brighton 925 Other
Brighton 2783 Police investigation procedure
Brighton 1627 Violence and harassment
Charlestown 200 Disputes
Charlestown 416 Drugs and disorderly conduct
Charlestown 428 Fraud and law violations
Charlestown 1671 Larceny and vandalism
Charlestown 338 Other
Charlestown 1162 Police investigation procedure
Charlestown 550 Violence and harassment
Dorchester 3066 Disputes
Dorchester 2587 Drugs and disorderly conduct
Dorchester 3705 Fraud and law violations
Dorchester 9230 Larceny and vandalism
Dorchester 2792 Other
Dorchester 7428 Police investigation procedure
Dorchester 4067 Violence and harassment
Downtown 254 Disputes
Downtown 2510 Drugs and disorderly conduct
Downtown 3245 Fraud and law violations
Downtown 9256 Larceny and vandalism
Downtown 2034 Other
Downtown 5464 Police investigation procedure
Downtown 3497 Violence and harassmen

In [164]:
dico

{'Brighton': [5274, 'Larceny and vandalism'],
 'Charlestown': [1671, 'Larceny and vandalism'],
 'Dorchester': [9230, 'Larceny and vandalism'],
 'Downtown': [9256, 'Larceny and vandalism'],
 'East Boston': [2911, 'Larceny and vandalism'],
 'Hyde Park': [3517, 'Larceny and vandalism'],
 'Jamaica Plain': [4546, 'Larceny and vandalism'],
 'Mattapan': [7001, 'Police investigation procedure'],
 'Roxbury': [10471, 'Larceny and vandalism'],
 'South Boston': [5637, 'Larceny and vandalism'],
 'South End': [14012, 'Larceny and vandalism'],
 'West Roxbury': [2915, 'Larceny and vandalism']}

In [167]:
most_crime = pd.DataFrame.from_dict(dico,orient='index')
most_crime

,0,1
Brighton,5274,Larceny and vandalism
Charlestown,1671,Larceny and vandalism
Dorchester,9230,Larceny and vandalism
Downtown,9256,Larceny and vandalism
East Boston,2911,Larceny and vandalism
Hyde Park,3517,Larceny and vandalism
Jamaica Plain,4546,Larceny and vandalism
Mattapan,7001,Police investigation procedure
Roxbury,10471,Larceny and vandalism
South Boston,5637,Larceny and vandalism


**❓ Can you tell what is the second most common offense int the Brighton district?**

In [121]:
second_most_common_offence = "Police investigation procedure"

### Average distance to the police station per district

Second, based on the Haversine distance function defined below, the head of the Police Department would like to know, for each district, **the average distance between the location of the incident and the police station**. 

**📝 Construct a DataFrame with one row per police district, designated by its full name, and one column that displays this average Haversine distance.**

In [129]:
# Haversine distance function
from math import radians, sin, cos, asin, sqrt

def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Compute distance (km) between two pairs of (lat, lng) coordinates
    See - (https://en.wikipedia.org/wiki/Haversine_formula)
    """
    
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    
    return 2 * 6371 * asin(sqrt(a))

In [131]:
# YOUR CODE HERE
data['distance'] =data.apply(lambda x: haversine_distance(x['LONG'], x['LAT'],x['LONG_POLICE_STATION'],x['LAT_POLICE_STATION']), axis=1) 

In [140]:
# YOUR CODE HERE
mean_distance = data[['NAME','distance']].groupby('NAME').mean()

In [141]:
mean_distance

,distance
NAME,
Brighton,3.770867
Charlestown,58.758394
Dorchester,3.174444
Downtown,6.336657
East Boston,5.568626
Hyde Park,2.397613
Jamaica Plain,2.585737
Mattapan,1.908636
Roxbury,2.989029


In [149]:
a_d = mean_distance.iloc[0][0]

**❓ Can you tell what is the average distance between the police station and the offenses in the Brighton district?**

In [150]:
# YOUR CODE HERE
average_distance_km = a_d

In [151]:
result = ChallengeResult('manipulation',
                         second_most_common_offence=second_most_common_offence,
                         average_distance_km=average_distance_km)
result.write()

# 3. Short presentation (REQUIRED TO VALIDATE THE CERTIFICATION)
🚨🚨🚨🚨🚨🚨

Using the latest dataset that you loaded, your mission is now to prepare 5 slides (including a title slide) that you would present to the head of the Police Department. You may or may not, as you prefer, include the output of the two "Further data manipulations" tasks in your presentation.

⚠️  You can use any presentation editor of your choice, but **the slides must be shared either in HTML or in PDF format and saved in the current directory**

Before you get started, here are four small pieces of advice:

- to prepare your data visualisations, do not hesitate to create a separate, blank notebook; 


- pay particular attention to the readability and the clarity of your legends, titles, charts and tables; 


- the Pandas `resample` method might be useful if you want to plot time trends;


- keep in mind that you are working with real data and sometimes, data are not very talkative. Do not feel discouraged if your charts do not provide revolutionary insights: typically, an absence of trend is a substantial piece of information!

Good luck in changing Boston residents' lives!

# A word of conclusion

Congratulations for going through the exercise 🎉

If you wish to pursue your analysis at some point, note that all datasets (and many others) are publicly available online, on the [Analyze Boston](https://data.boston.gov) website.

Besides, if you are interested in the topic, you can start by reading the work of Nobel Prize laureate Gary Becker, who was the first to model crime as a rational phenomenon, similarly to an economic decision. This model, although it has limitations, marked a breakthrough in the study of crime and paved the way for many empirical studies that further analysed the socio-economic determinants of illegal activities. 

👉 [Link](https://olis.leg.state.or.us/liz/2017R1/Downloads/CommitteeMeetingDocument/125036) to download a full-text version of "Crime and Punishment: An Economic Approach" by Becker (1968)